## Analysing Maximum Likelihood Estimation & Maximum A Posterior Estimation

### Bayes Theorem
The Bayes Theorem is an important theorem in probability. It is used in calculating inverse probability. Let A and B be two events. A|B denotes that event A happens given that event B has happened. Using Bayes Theorem, we can say that:<br>
<center><h3><i>P(A|B)=P(B|A)*P(A)/P(B)</i></h3></center><br>
This theorem forms the foundation of numerous Machine Learning algorithms.

### Introduction to MLE
Maximum likelihood estimation is a method that determines values for the parameters of a model. The parameter values are found such that they maximise the likelihood that the process described by the model produced the data that were actually observed. In class we studied the use of MLE in finding best parameters for Linear Regression. <em>Here we are using it in a general purpose common sense type of problem.</em>

### Enter MAP

MLE can be viewed as MLE with prior knowledge. Using prior knowledge we can improve our estimation. Thus, MAP is more likely to give higher accuracy.

### The Problem Statement
We have a bag of apples. A random apple is drawn from the bag based on the probabilities of a discrete Gaussian distribution. We know the true weight of the apple. However, we are weighing it using a broken scale that gives a randomly generated error sampled from a Gaussian distribution. We can take measurements any number of times. We will be using MLE and MAP to see how close to the actual measurement we get.

### Step 0: Importing Modules, Defining Useful Functions and Initialising Some Data

In [2]:
import numpy as np
pi=np.pi
mean_weights=85
variance_weights=20

def gaussian_probability_weight(point):
    return (1/(np.pi*2*variance_weights)**0.5)*np.exp(-((point-mean_weights)**2)/(2*variance_weights))

mean_error=0
variance_error=5

def gaussian_probability_errors(point):
    return (1/(np.pi*2*variance_error)**0.5)*np.exp(-((point-mean_error)**2)/(2*variance_error))

### Step 1: Getting Priors Right

In [3]:
w={}
e={}
i=4
for i in range(30,301):
    w[i]=gaussian_probability_weight(i)
for i in range(-11,12):
    e[i]=gaussian_probability_errors(i)
sum_w=0
for i in w.keys():
    sum_w+=w[i]
sum_e=0
for i in e.keys():
    sum_e+=e[i]
for i in w.keys():
    w[i]=w[i]/sum_w
for i in e.keys():
    e[i]=e[i]/sum_e

### Step 2: Function that Controls the Game

In [4]:
measures=3
def get_measure(x):
    measurements=[]
    for i in range(measures):
        measurements.append(x+np.random.choice([i for i in e.keys()],p=[e[i] for i in e.keys()]))
    return measurements

### Step 3: Let the Guessing Begin

In [17]:
t=np.random.choice([i for i in w.keys()],p=[w[i] for i in w.keys()])
measurements=get_measure(t)
map_measure=[0,0]
mle_measure=[0,0]
weights_exp=[i for i in w.keys()]
for i in range(len(weights_exp)):
        true_weight=weights_exp[i]
        p_true_weight=w[true_weight]
        p_liklihood=1
        for j in range(len(measurements)):
            error=measurements[j]-true_weight
            if error in e.keys():
                p_liklihood*=e[error]
            else:
                p_liklihood=0
            if p_liklihood>mle_measure[0]:
                mle_measure=[p_liklihood,weights_exp[i]]
            p_map=p_liklihood*(w[weights_exp[i]])
            if p_map>map_measure[0]:
                map_measure=[p_map,weights_exp[i]]
print(f"Average measurement: {np.mean(measurements).round(1)}")
print(f"Maximum Liklihood estimate: {mle_measure[1]}")
print(f"Maximum A Posterior estimate: {map_measure[1]}")
print(f"The true weight of the apple was: {t}")

Average measurement: 83.0
Maximum Liklihood estimate: 81
Maximum A Posterior estimate: 82
The true weight of the apple was: 83


### Step 4: Different Views on Accuracy

In [74]:
answers=[]
for i in range(1000):
        t=np.random.choice([i for i in w.keys()],p=[w[i] for i in w.keys()])
        #perform measurements
        measurements=get_measure(t)
        map_measure=[0,0]
        mle_measure=[0,0]
        weights_exp=[i for i in w.keys()]
        for i in range(len(weights_exp)):
            true_weight=weights_exp[i]
            p_true_weight=w[true_weight]
            p_liklihood=1
            for j in range(len(measurements)):
                error=measurements[j]-true_weight
                if error in e.keys():
                    p_liklihood*=e[error]
                else:
                    p_liklihood=0
            if p_liklihood>mle_measure[0]:
                mle_measure=[p_liklihood,weights_exp[i]]
            p_map=p_liklihood*(w[weights_exp[i]])
            if p_map>map_measure[0]:
                map_measure=[p_map,weights_exp[i]]
        answers.append((t,map_measure[1],mle_measure[1]))
map_acc=0
mle_acc=0
for i in answers:
        if abs(i[0]-i[1])<=abs(i[0]-i[2]):
            map_acc+=1
        if abs(i[0]-i[1])>=abs(i[0]-i[2]):
            mle_acc+=1
print("Comparing accuracy based on who is closer to the observed value in each case:")
print(f"MAP: {map_acc/1000}")
print(f"MLE: {mle_acc/1000}")
print("Comparison with Square Error as a Metric:")
map_acc=0
mle_acc=0
for i in answers:
        map_acc+=(i[1]-i[0])**2
        mle_acc+=(i[2]-i[0])**2
print(f"MLE: {mle_acc}")
print(f"MAP: {map_acc}")

Comparing accuracy based on who is closer to the observed value in each case:
MAP: 0.866
MLE: 0.819
Comparison with Square Error as a Metric:
MLE: 1724
MAP: 1596
